1. Read in raw data file
2. Check description of events
3. Filter events and trial number based on condition
4. Check sum of saccades for different conditions
5. Check interperson differences and averages
6. Find out how to calculate saccades in visual degrees from x and y positiions


In [60]:
from mne.io import read_raw_eyelink
from mne.preprocessing.eyetracking import read_eyelink_calibration
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from pprint import pprint

In [31]:
path = '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc'
asc_files = glob.glob(os.path.join(path, "*.asc"))

In [32]:
asc_files

['/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/75.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/76.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/77.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/83.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/82.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/81.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/85.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/84.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/78.asc',
 '/Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/87.asc']

In [33]:
raw_dict = {}
calibration_dict = {}

for path in asc_files:
    filename = os.path.basename(path)
    number_str = os.path.splitext(filename)[0]
    
    raw = read_raw_eyelink(path, create_annotations=True)
    df = raw.annotations.to_data_frame()
    
    calibration = read_eyelink_calibration(
        path,
        screen_resolution=(1920, 1080),
        screen_size=(0.53, 0.3),
        screen_distance=0.9,
    )[0]
    
    # Store in dictionary
    raw_dict[number_str] = {
        'raw': raw,
        'df': df
    }
    
    calibration_dict[f'calibration_{number_str}'] = calibration
    
# Check the participant number included
raw_dict.keys()
calibration_dict.keys()


Loading /Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/75.asc
Pixel coordinate data detected.Pass `scalings=dict(eyegaze=1e3)` when using plot method to make traces more legible.
Pupil-size diameter detected.
There are 841 recording blocks in this file. Times between blocks will be annotated with BAD_ACQ_SKIP.
Reading calibration data from /Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/75.asc
Loading /Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/76.asc
Pixel coordinate data detected.Pass `scalings=dict(eyegaze=1e3)` when using plot method to make traces more legible.
Pupil-size diameter detected.
There are 1200 recording blocks in this file. Times between blocks will be annotated with BAD_ACQ_SKIP.
Reading calibration data from /Users/wouter/Documents/phd/projects/supervision/harry/7LineSegments/EyeLink/asc/76.asc
Loading /Users/wouter/Documents/phd/projects/supervision/harry/7LineSeg

dict_keys(['calibration_75', 'calibration_76', 'calibration_77', 'calibration_83', 'calibration_82', 'calibration_81', 'calibration_85', 'calibration_84', 'calibration_78', 'calibration_87'])

In [54]:
print(raw_dict['82']['df'].head(10))
print('Descriptions:')
print(set(raw_dict['82']['df']['description']))

                    onset  duration                          description  \
0 2025-05-12 08:43:45.007     0.365                             fixation   
1 2025-05-12 08:43:45.030     0.000  TRIAL_START 0 | FarHighNarrow | 0.1   
2 2025-05-12 08:43:45.372     0.016                              saccade   
3 2025-05-12 08:43:45.388     1.437                             fixation   
4 2025-05-12 08:43:46.256     0.000    TRIAL_END 0 | FarHighNarrow | 0.1   
5 2025-05-12 08:43:46.826     0.023                         BAD_ACQ_SKIP   
6 2025-05-12 08:43:46.856     1.819                             fixation   
7 2025-05-12 08:43:46.880     0.000   TRIAL_START 1 | FarLowNarrow | 0.1   
8 2025-05-12 08:43:48.064     0.000     TRIAL_END 1 | FarLowNarrow | 0.1   
9 2025-05-12 08:43:48.676     0.023                         BAD_ACQ_SKIP   

                             ch_names  
0  (xpos_left, ypos_left, pupil_left)  
1                                  ()  
2  (xpos_left, ypos_left, pupil_left)  
3  

In [ ]:
all_summary_stats = []

for participant_id, participant_data in raw_dict.items():
    annotations_df = participant_data['df']
    
    # Filter for saccades
    saccades_df = annotations_df[
        annotations_df['description'].str.contains('saccade', case=False, na=False)
    ]
    
    # Collect summary statistics
    summary_stats = {
        "Participant": participant_id,
        "Total saccades": len(saccades_df),
        "Mean duration (s)": saccades_df['duration'].mean(),
        "Min duration (s)": saccades_df['duration'].min(),
        "Max duration (s)": saccades_df['duration'].max(),
        "Unique descriptions": saccades_df['description'].nunique()
    }
    
    all_summary_stats.append(summary_stats)

# Convert the list of dicts into a DataFrame
summary_df = pd.DataFrame(all_summary_stats)

# Display the summary
display(summary_df) 

,Participant,Total saccades,Mean duration (s),Min duration (s),Max duration (s),Unique descriptions
0,75,2152,0.130776,0.005,67.985,1
1,76,2852,0.129629,0.005,1.839,1
2,77,2387,0.028446,0.005,0.489,1
3,83,2801,0.107695,0.005,1.816,1
4,82,2960,0.017446,0.005,0.258,1
5,81,1161,0.072834,0.005,0.254,1
6,85,2840,0.064345,0.005,0.422,1
7,84,2572,0.048308,0.005,0.839,1
8,78,3698,0.080674,0.005,0.744,1
9,87,2906,0.070091,0.005,0.449,1


In [65]:
# Conditions
conditions = ["constant", "ThreeLinesControl", "FarHighNarrow", "FarHighWide"]

# Collect per-participant condition counts
all_counts = []

for participant_id, participant_data in raw_dict.items():
    annotation_df = participant_data['df']
    
    participant_counts = {"Participant": participant_id}
    
    for condition in conditions:
        count = annotation_df['description'].str.contains(condition, case=False, na=False).sum()
        participant_counts[condition] = count
    
    all_counts.append(participant_counts)

# Create a summary DataFrame
condition_summary_df = pd.DataFrame(all_counts)
condition_summary_df.set_index("Participant", inplace=True)

# Display the result
display(condition_summary_df)


,constant,ThreeLinesControl,FarHighNarrow,FarHighWide
Participant,,,,
75,253,268,250,259
76,387,375,379,382
77,398,397,399,395
83,368,371,375,371
82,395,398,397,399
81,366,386,375,372
85,400,400,400,400
84,389,394,395,396
78,374,372,366,372
